
# Exercise 5 — Ontology & Reasoning (Conference Organization)
**Course:** Web Semantics & Linked Data** · **Tools:** Protégé (+ HermiT / Fact++)



## Goals (concise)
- Model a small domain as an ontology (classes, properties, constraints).
- Use a reasoner to check consistency and inferred knowledge.
- Reflect on mapping English rules to OWL axioms.



## 1) Warm-up (5 minutes)
Write **three** English statements (rules) about conferences that should always hold.

1. Every conference must have at least one organizer.
2. Every conference must take place at exactly one location.
3. No person can be both a speaker and an organizer of the same conference.



## 2) Part I — Model the domain (15–20 minutes)
In **Protégé**:
- Add essential **classes** (people, events, places, etc.).
- Add **object**/**datatype properties** (set **domain**/**range**).
- Create at least **two individuals** and link them.



## 3) Part II — Add constraints (15–20 minutes)
Add appropriate **cardinality**, **disjointness**, and optionally **equivalence** axioms.  
*(Briefly justify your choices below.)*



### Notes on my modeling choices
- …
- …
- …



## 4) Part III — Reasoning (10–15 minutes)
Run a reasoner (HermiT/Fact++) in Protégé and note:
- **One inferred classification**

Web_Semantic organizedBy Rute (inverse property that wasnt defined)
Ontology conference Web_Semantic (inverse property that wasnt defined)

- **One issue/inconsistency** (if any) and how you resolved it

Kairi (attendee) is being forced to belong to Speaker
-> Changed the attends from Attendee and Speaker to Person; Made attends disjoint with Organizes




## (Optional) Save a starter ontology (TTL)
Run the next cell to write `conference_ontology_starter.ttl` in this folder, then open it in **Protégé**.


In [1]:
# Writes 'conference_ontology_starter.ttl' to the current directory
ttl_path = "conference_ontology_starter.ttl"
ttl_content = r"""@prefix rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix owl:  <http://www.w3.org/2002/07/owl#> .
@prefix xsd:  <http://www.w3.org/2001/XMLSchema#> .
@prefix conf: <http://example.org/conference#> .

conf: a owl:Ontology ;
  rdfs:label "Conference Organization Ontology (starter)" .

conf:Person a owl:Class .
conf:Speaker a owl:Class ; rdfs:subClassOf conf:Person .
conf:Attendee a owl:Class ; rdfs:subClassOf conf:Person .
conf:Organizer a owl:Class ; rdfs:subClassOf conf:Person .

conf:Conference a owl:Class .
conf:Track a owl:Class .
conf:Session a owl:Class .
conf:Talk a owl:Class ; rdfs:subClassOf conf:Session .
conf:Workshop a owl:Class ; rdfs:subClassOf conf:Session .
conf:Tutorial a owl:Class ; rdfs:subClassOf conf:Session .

conf:Room a owl:Class .
conf:Building a owl:Class .

[] a owl:AllDisjointClasses ; owl:members ( conf:Room conf:Building conf:Session conf:Conference conf:Person ) .
conf:Speaker owl:disjointWith conf:Attendee .

conf:hasSpeaker a owl:ObjectProperty ; rdfs:domain conf:Session ; rdfs:range conf:Speaker .
conf:givesTalk a owl:ObjectProperty ; owl:inverseOf conf:hasSpeaker ; rdfs:domain conf:Speaker ; rdfs:range conf:Session .
conf:hasOrganizer a owl:ObjectProperty ; rdfs:domain conf:Conference ; rdfs:range conf:Organizer .
conf:organizes a owl:ObjectProperty ; owl:inverseOf conf:hasOrganizer ; rdfs:domain conf:Organizer ; rdfs:range conf:Conference .
conf:takesPlaceIn a owl:ObjectProperty , owl:FunctionalProperty ; rdfs:domain conf:Session ; rdfs:range conf:Room .
conf:locatedIn a owl:ObjectProperty ; rdfs:domain conf:Room ; rdfs:range conf:Building .
conf:inTrack a owl:ObjectProperty ; rdfs:domain conf:Session ; rdfs:range conf:Track .
conf:partOf a owl:ObjectProperty ; rdfs:domain conf:Session ; rdfs:range conf:Conference .
conf:registeredFor a owl:ObjectProperty ; rdfs:domain conf:Person ; rdfs:range conf:Conference .

conf:hasName a owl:DatatypeProperty ; rdfs:domain owl:Thing ; rdfs:range xsd:string .
conf:hasStartTime a owl:DatatypeProperty ; rdfs:domain conf:Session ; rdfs:range xsd:dateTime .

conf:Session rdfs:subClassOf [
  a owl:Restriction ; owl:onProperty conf:takesPlaceIn ; owl:cardinality "1"^^xsd:nonNegativeInteger
] , [
  a owl:Restriction ; owl:onProperty conf:hasSpeaker ; owl:minCardinality "1"^^xsd:nonNegativeInteger
] .

conf:Conference rdfs:subClassOf [
  a owl:Restriction ; owl:onProperty conf:hasOrganizer ; owl:minCardinality "1"^^xsd:nonNegativeInteger
] .

conf:Room rdfs:subClassOf [
  a owl:Restriction ; owl:onProperty conf:locatedIn ; owl:cardinality "1"^^xsd:nonNegativeInteger
] .

conf:Attendee owl:equivalentClass [
  a owl:Class ;
  owl:intersectionOf ( conf:Person [ a owl:Restriction ; owl:onProperty conf:registeredFor ; owl:someValuesFrom conf:Conference ] )
] .

conf:BuildingA a conf:Building ; conf:hasName "Main Building A" .
conf:Room101 a conf:Room ; conf:hasName "Room 101" ; conf:locatedIn conf:BuildingA .

conf:Conf2025 a conf:Conference ; conf:hasName "Semantic Web Conf 2025" .

conf:Alice_Wong a conf:Organizer , conf:Person ; conf:hasName "Alice Wong" ; conf:organizes conf:Conf2025 .
conf:John_Smith a conf:Speaker , conf:Person ; conf:hasName "John Smith" ; conf:registeredFor conf:Conf2025 .
conf:Maria_Garcia a conf:Person ; conf:hasName "Maria Garcia" ; conf:registeredFor conf:Conf2025 .

conf:Keynote_Opening a conf:Talk , conf:Session ;
  conf:hasName "Opening Keynote" ;
  conf:hasSpeaker conf:John_Smith ;
  conf:takesPlaceIn conf:Room101 ;
  conf:partOf conf:Conf2025 .
"""
with open(ttl_path, "w", encoding="utf-8") as f:
    f.write(ttl_content)
print("Wrote:", ttl_path)

Wrote: conference_ontology_starter.ttl


In [2]:
# Quick check
import os
print("Exists:", os.path.exists("conference_ontology_starter.ttl"))
if os.path.exists("conference_ontology_starter.ttl"):
    print("Size (bytes):", os.path.getsize("conference_ontology_starter.ttl"))


Exists: True
Size (bytes): 3643



## Optional — Explore with `rdflib` (local reasoning not required)
> You need `rdflib` installed in your environment. If not present, run in a terminal:  
> `pip install rdflib`
> if using `uv`:
> `uv sync`


In [3]:
# Try loading the TTL with rdflib and count triples
try:
    from rdflib import Graph
    g = Graph()
    g.parse("conference_ontology_starter.ttl", format="turtle")
    print("Loaded triples:", len(g))
except ImportError:
    print("rdflib not installed. Install with: pip install rdflib")


Loaded triples: 119
